# A model of production and transportation

In [1]:
# install dependencies
%pip install -q amplpy pandas

from amplpy import AMPL, ampl_notebook
import pandas as pd

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics

Note: you may need to restart the kernel to use updated packages.


Large linear programs can be created not only by tying together
small models of one kind, as in the two examples above, but by linking
different kinds of models.  We conclude this chapter with an example that combines
features of both production and transportation models.

Suppose that the steel products are made
at several mills, from which they are shipped to customers at the various
factories.  For each mill we can define a separate production
model to optimize the amounts of each product to make.
For each product we can define a separate
transportation model, with mills as origins and factories as
destinations, to optimize the amounts of the product to be
shipped.  We would like to link all these separate models into a
single integrated model of production and transportation.

To begin, we replicate the production model
of Figure @Tut1@-@steel.mod@ over mills &ndash; that is,
origins &ndash; rather than over weeks as in the previous example:
```
set PROD;   # products
set ORIG;   # origins (steel mills)

param rate {ORIG,PROD} > 0;  # tons per hour at origins
param avail {ORIG} >= 0;     # hours available at origins

var Make {ORIG,PROD} >= 0;   # tons produced at origins

subject to Time {i in ORIG}:
   sum {p in PROD} (1/rate[i,p]) * Make[i,p] <= avail[i];
```
We have temporarily dropped the components pertaining to the objective,
to which we will return later.  We have also dropped the market demand
parameters, since the demands are now properly associated with the
destinations in the transportation models.

The next step is to replicate the transportation model,
Figure @Tut3@-@transp.mod@, over products,
as we did in the multicommodity example at the beginning of this
chapter:
```
set ORIG;   # origins (steel mills)
set DEST;   # destinations (factories)
set PROD;   # products

param supply {ORIG,PROD} >= 0; # tons available at origins
param demand {DEST,PROD} >= 0; # tons required at destinations

var Trans {ORIG,DEST,PROD} >= 0; # tons shipped

subject to Supply {i in ORIG, p in PROD}:
   sum {j in DEST} Trans[i,j,p] = supply[i,p];

subject to Demand {j in DEST, p in PROD}:
   sum {i in ORIG} Trans[i,j,p] = demand[j,p];
```
Comparing the resulting production and transportation models, we see
that the sets of origins
`ORIG` ) (
and products
`PROD` ) (
are the same in both models.
Moreover, the "tons available at origins"
`supply` ) (
in the transportation
model are really the same thing as the "tons produced at origins"
`Make` ) (
in
the production model, since the steel available for shipping will be
whatever is made at the mill.

We can thus merge the two models, dropping the
definition of
`supply`
and substituting
`Make[i,p]`
for the occurrence of
`supply[i,p]` :
```
subject to Supply {i in ORIG, p in PROD}:
  sum {j in DEST} Trans[i,j,p] = Make[i,p];
```
There are several ways in which we might add an objective to
complete the model.  Perhaps the simplest is to define a cost per ton
corresponding to each variable.  We define a parameter
`make_cost`
so
that there is a term
`make_cost[i,p]
*
Make[i,p]`
in the objective for
each origin
`i`
and product
`p` ;
and we define
`trans_cost`
so that there is
a term
`trans_cost[i,j,p]
*
Trans[i,j,p]`
in the objective for each
origin
`i` ,
destination
`j`
and product
`p` .
The full model is
shown in Figure @Tut4@-@steelP.mod@.
.F1
``` 0
.get steelP.mod
```
.C "Figure @Tut4@-@steelP.mod@" "Production/transportation model, 3rd version (\f(CWsteelP.mod\fP).
.ix file~[steelP.mod]
.F2
.F1
```
.get steelP.dat
```
.C "Figure @Tut4@-@steelP.dat@" "Data for production/transportation model (\f(CWsteelP.dat\fP).
.ix file~[steelP.dat]
.F2

In [2]:
%%writefile steelP.mod

set ORIG;   # origins (steel mills)
set DEST;   # destinations (factories)
set PROD;   # products

param rate {ORIG,PROD} > 0;     # tons per hour at origins
param avail {ORIG} >= 0;        # hours available at origins
param demand {DEST,PROD} >= 0;  # tons required at destinations

param make_cost {ORIG,PROD} >= 0;        # manufacturing cost/ton
param trans_cost {ORIG,DEST,PROD} >= 0;  # shipping cost/ton

var Make {ORIG,PROD} >= 0;       # tons produced at origins
var Trans {ORIG,DEST,PROD} >= 0; # tons shipped

minimize Total_Cost:
   sum {i in ORIG, p in PROD} make_cost[i,p] * Make[i,p] +
   sum {i in ORIG, j in DEST, p in PROD}
            trans_cost[i,j,p] * Trans[i,j,p];

subject to Time {i in ORIG}:
   sum {p in PROD} (1/rate[i,p]) * Make[i,p] <= avail[i];

subject to Supply {i in ORIG, p in PROD}:
   sum {j in DEST} Trans[i,j,p] = Make[i,p];

subject to Demand {j in DEST, p in PROD}:
   sum {i in ORIG} Trans[i,j,p] = demand[j,p];

Overwriting steelP.mod


In [3]:
ORIG = ['GARY', 'CLEV', 'PITT']
DEST = ['FRA', 'DET', 'LAN', 'WIN', 'STL', 'FRE', 'LAF']
PROD = ['bands', 'coils', 'plate']

avail =  {'GARY': 20, 'CLEV': 15, 'PITT': 20}

"""
param demand (tr):
             FRA   DET   LAN   WIN   STL   FRE   LAF :=
    bands    300   300   100    75   650   225   250
    coils    500   750   400   250   950   850   500
    plate    100   100     0    50   200   100   250 ;
"""

df_demand = pd.DataFrame(
    [        
        ['bands', 300, 300, 100, 75, 650, 225, 250],
        ['coils', 500, 750, 400, 250, 950, 850, 500],
        ['plate', 100, 100, 0, 50, 200, 100, 250]
    ],
    columns = ['PROD', 'FRA', 'DET', 'LAN', 'WIN', 'STL', 'FRE', 'LAF']
).set_index('PROD')


display(df_demand)
    
"""    
param rate (tr):  GARY   CLEV   PITT :=
          bands    200    190    230
          coils    140    130    160
          plate    160    160    170 ;
"""

df_rate = pd.DataFrame(
    [        
        ['bands', 200, 190, 230],
        ['coils', 140, 130, 160],
        ['plate', 160, 160, 170]
    ],
    columns = ['PROD', 'GARY', 'CLEV', 'PITT']
).set_index('PROD')

"""
param make_cost (tr):
                  GARY   CLEV   PITT :=
          bands    180    190    190
          coils    170    170    180
          plate    180    185    185 ;
"""

df_make_cost = pd.DataFrame(
    [
        ['bands', 180, 190, 190],
        ['coils', 170, 170, 180],
        ['plate', 180, 185, 185]
    ],
    columns = ['PROD', 'GARY', 'CLEV', 'PITT']
).set_index('PROD')

"""
param trans_cost :=

 [*,*,bands]:  FRA  DET  LAN  WIN  STL  FRE  LAF :=
        GARY    30   10    8   10   11   71    6
        CLEV    22    7   10    7   21   82   13
        PITT    19   11   12   10   25   83   15

 [*,*,coils]:  FRA  DET  LAN  WIN  STL  FRE  LAF :=
        GARY    39   14   11   14   16   82    8
        CLEV    27    9   12    9   26   95   17
        PITT    24   14   17   13   28   99   20

 [*,*,plate]:  FRA  DET  LAN  WIN  STL  FRE  LAF :=
        GARY    41   15   12   16   17   86    8
        CLEV    29    9   13    9   28   99   18
        PITT    26   14   17   13   31  104   20 ;
"""

"""
mydict = {
('GARY', 'FRA', 'bands') : 30 ,
('GARY', 'DET', 'bands') : 10 ,
('GARY', 'LAN', 'bands') : 8 ,
('GARY', 'WIN', 'bands') : 10 ,
('GARY', 'STL', 'bands') : 11 ,
('GARY', 'FRE', 'bands') : 71 ,
('GARY', 'LAF', 'bands') : 6 ,
('CLEV', 'FRA', 'bands') : 22 ,
('CLEV', 'DET', 'bands') : 7 ,
('CLEV', 'LAN', 'bands') : 10 ,
('CLEV', 'WIN', 'bands') : 7 ,
('CLEV', 'STL', 'bands') : 21 ,
('CLEV', 'FRE', 'bands') : 82 ,
('CLEV', 'LAF', 'bands') : 13 ,
('PITT', 'FRA', 'bands') : 19 ,
('PITT', 'DET', 'bands') : 11 ,
('PITT', 'LAN', 'bands') : 12 ,
('PITT', 'WIN', 'bands') : 10 ,
('PITT', 'STL', 'bands') : 25 ,
('PITT', 'FRE', 'bands') : 83 ,
('PITT', 'LAF', 'bands') : 15 ,
('GARY', 'FRA', 'coils') : 39 ,
('GARY', 'DET', 'coils') : 14 ,
('GARY', 'LAN', 'coils') : 11 ,
('GARY', 'WIN', 'coils') : 14 ,
('GARY', 'STL', 'coils') : 16 ,
('GARY', 'FRE', 'coils') : 82 ,
('GARY', 'LAF', 'coils') : 8 ,
('CLEV', 'FRA', 'coils') : 27 ,
('CLEV', 'DET', 'coils') : 9 ,
('CLEV', 'LAN', 'coils') : 12 ,
('CLEV', 'WIN', 'coils') : 9 ,
('CLEV', 'STL', 'coils') : 26 ,
('CLEV', 'FRE', 'coils') : 95 ,
('CLEV', 'LAF', 'coils') : 17 ,
('PITT', 'FRA', 'coils') : 24 ,
('PITT', 'DET', 'coils') : 14 ,
('PITT', 'LAN', 'coils') : 17 ,
('PITT', 'WIN', 'coils') : 13 ,
('PITT', 'STL', 'coils') : 28 ,
('PITT', 'FRE', 'coils') : 99 ,
('PITT', 'LAF', 'coils') : 20 ,
('GARY', 'FRA', 'plate') : 41 ,
('GARY', 'DET', 'plate') : 15 ,
('GARY', 'LAN', 'plate') : 12 ,
('GARY', 'WIN', 'plate') : 16 ,
('GARY', 'STL', 'plate') : 17 ,
('GARY', 'FRE', 'plate') : 86 ,
('GARY', 'LAF', 'plate') : 8 ,
('CLEV', 'FRA', 'plate') : 29 ,
('CLEV', 'DET', 'plate') : 9 ,
('CLEV', 'LAN', 'plate') : 13 ,
('CLEV', 'WIN', 'plate') : 9 ,
('CLEV', 'STL', 'plate') : 28 ,
('CLEV', 'FRE', 'plate') : 99 ,
('CLEV', 'LAF', 'plate') : 18 ,
('PITT', 'FRA', 'plate') : 26 ,
('PITT', 'DET', 'plate') : 14 ,
('PITT', 'LAN', 'plate') : 17 ,
('PITT', 'WIN', 'plate') : 13 ,
('PITT', 'STL', 'plate') : 31 ,
('PITT', 'FRE', 'plate') : 104 ,
('PITT', 'LAF', 'plate') : 20 ,
}

print(mydict)


for i in ORIG:
    for j in DEST:
        k1 = "'" + i + "'"
        k2 = "'" + j + "'"
        print(k1, k2, mydict[(i, j, 'bands')], mydict[(i, j, 'coils')], mydict[(i, j, 'plate')])
"""

df_trans_cost = pd.DataFrame(
    [
        ['GARY', 'FRA', 30, 39, 41],
        ['GARY', 'DET', 10, 14, 15],
        ['GARY', 'LAN', 8, 11, 12],
        ['GARY', 'WIN', 10, 14, 16],
        ['GARY', 'STL', 11, 16, 17],
        ['GARY', 'FRE', 71, 82, 86],
        ['GARY', 'LAF', 6, 8, 8],
        ['CLEV', 'FRA', 22, 27, 29],
        ['CLEV', 'DET', 7, 9, 9],
        ['CLEV', 'LAN', 10, 12, 13],
        ['CLEV', 'WIN', 7, 9, 9],
        ['CLEV', 'STL', 21, 26, 28],
        ['CLEV', 'FRE', 82, 95, 99],
        ['CLEV', 'LAF', 13, 17, 18],
        ['PITT', 'FRA', 19, 24, 26],
        ['PITT', 'DET', 11, 14, 14],
        ['PITT', 'LAN', 12, 17, 17],
        ['PITT', 'WIN', 10, 13, 13],
        ['PITT', 'STL', 25, 28, 31],
        ['PITT', 'FRE', 83, 99, 104],
        ['PITT', 'LAF', 15, 20, 20]
    ],
    columns = ['ORIG', 'DEST', 'bands', 'coils', 'plate']
).set_index(['ORIG', 'DEST'])

display(df_trans_cost)

,FRA,DET,LAN,WIN,STL,FRE,LAF
PROD,,,,,,,
bands,300,300,100,75,650,225,250
coils,500,750,400,250,950,850,500
plate,100,100,0,50,200,100,250


bands  coils  plate
ORIG DEST                     
GARY FRA      30     39     41
     DET      10     14     15
     LAN       8     11     12
     WIN      10     14     16
     STL      11     16     17
     FRE      71     82     86
     LAF       6      8      8
CLEV FRA      22     27     29
     DET       7      9      9
     LAN      10     12     13
     WIN       7      9      9
     STL      21     26     28
     FRE      82     95     99
     LAF      13     17     18
PITT FRA      19     24     26
     DET      11     14     14
     LAN      12     17     17
     WIN      10     13     13
     STL      25     28     31
     FRE      83     99    104
     LAF      15     20     20

Reviewing this formulation, we might observe that, according to the
`Supply`
declaration, the nonnegative expression
```
sum {j in DEST} Trans[i,j,p]
```
can be
substituted for
`Make[i,p]` .
If we make this substitution for all occurrences
of
`Make[i,p]`
in the objective and in the
`Time`
constraints, we no
longer need to include the
`Make`
variables or the
`Supply`
constraints in our
model, and our linear programs will be smaller as a result.  Nevertheless,
in most cases we will be better off leaving the model as it is shown above.
By "substituting out" the
`Make`
variables we render the model harder to
read, and not a great deal easier to solve.

As an instance of solving a linear program based on this model,
we can adapt the data from Figure @Tut4@-@multi.dat@, as
shown in Figure @Tut4@-@steelP.dat@.  Here are some representative
result values:


In [14]:
ampl = AMPL()
ampl.read('steelP.mod')

ampl.set['ORIG'] = ORIG
ampl.set['DEST'] = DEST
ampl.set['PROD'] = PROD

ampl.param['avail'] = avail

ampl.param['demand'] = df_demand.T
ampl.param['rate'] = df_rate.T
ampl.param['make_cost'] = df_make_cost.T

ampl.param['trans_cost'] = df_trans_cost

ampl.option['solver'] = 'highs'
ampl.solve()

df_make = ampl.var['Make'].to_pandas()
df_make.columns = ["Make"]
df_make.index.names = ['ORIG', 'PROD']
df_make = df_make.unstack()
display(df_make)

df_trans = ampl.var['Trans'].to_pandas()
df_trans.columns = ["Trans"]
df_trans.index.names = ['ORIG', 'DEST', 'PROD']

display(df_trans.unstack())
display(df_trans.loc[(df_trans!=0).any(axis=1)])


df_test = df_trans.unstack()
display(df_test.loc[(df_test!=0).any(axis=1)])

HiGHS 1.6.0:HiGHS 1.6.0: optimal solution; objective 1392175
27 simplex iterations
0 barrier iterations
 


Make            
PROD bands coils plate
ORIG                  
CLEV     0  1950     0
GARY  1125  1750   300
PITT   775   500   500

Trans            
PROD      bands coils plate
ORIG DEST                  
CLEV DET      0   750     0
     FRA      0     0     0
     FRE      0     0     0
     LAF      0   500     0
     LAN      0   400     0
     STL      0    50     0
     WIN      0   250     0
GARY DET      0     0     0
     FRA      0     0     0
     FRE    225   850   100
     LAF    250     0     0
     LAN      0     0     0
     STL    650   900   200
     WIN      0     0     0
PITT DET    300     0   100
     FRA    300   500   100
     FRE      0     0     0
     LAF      0     0   250
     LAN    100     0     0
     STL      0     0     0
     WIN     75     0    50

Trans
ORIG DEST PROD        
CLEV DET  coils    750
     LAF  coils    500
     LAN  coils    400
     STL  coils     50
     WIN  coils    250
GARY FRE  bands    225
          coils    850
          plate    100
     LAF  bands    250
     STL  bands    650
          coils    900
          plate    200
PITT DET  bands    300
          plate    100
     FRA  bands    300
          coils    500
          plate    100
     LAF  plate    250
     LAN  bands    100
     WIN  bands     75
          plate     50

Trans            
PROD      bands coils plate
ORIG DEST                  
CLEV DET      0   750     0
     LAF      0   500     0
     LAN      0   400     0
     STL      0    50     0
     WIN      0   250     0
GARY FRE    225   850   100
     LAF    250     0     0
     STL    650   900   200
PITT DET    300     0   100
     FRA    300   500   100
     LAF      0     0   250
     LAN    100     0     0
     WIN     75     0    50

```
ampl: \f(CXmodel steelP.mod; data steelP.dat; solve;\fP
CPLEX 8.0.0: optimal solution; objective 1392175
27 dual simplex iterations (0 in phase I)
```
```
ampl: \f(CXoption display_1col 5;\fP
ampl: \f(CXoption omit_zero_rows 1, omit_zero_cols 1;\fP
```
```
ampl: \f(CXdisplay Make;\fP
Make [*,*]
:     bands  coils plate    :=
CLEV      0   1950     0
GARY   1125   1750   300
PITT    775    500   500
;
```
```
ampl: \f(CXdisplay Trans;\fP
Trans [CLEV,*,*]
:   coils    :=
DET   750
LAF   500
LAN   400
STL    50
WIN   250
```
```
 [GARY,*,*]
:   bands coils plate    :=
FRE   225   850   100
LAF   250     0     0
STL   650   900   200
```
```
 [PITT,*,*]
:   bands coils plate    :=
DET   300     0   100
FRA   300   500   100
LAF     0     0   250
LAN   100     0     0
WIN    75     0    50
;
```
```
ampl: \f(CXdisplay Time;\fP
Time [*] :=
CLEV  -1300
GARY  -2800
;
```
As one might expect, the optimal solution does not ship all products from
all mills to all factories.
We have used the options
`omit_zero_rows`
and
`omit_zero_cols`
to suppress the printing of
table rows and columns that are all zeros.  The dual values
for
`Time`
show that additional capacity is likely to have the greatest impact on total
cost if it is placed at \s-2GARY\s0, and no impact if it is placed at \s-2PITT\s0.

We can also investigate the relative costs of production and shipping,
which are the two components of the objective:
```
ampl: \f(CXdisplay sum {i in ORIG, p in PROD}
            make_cost[i,p] * Make[i,p];\fP
sum{i in ORIG, p in PROD} make_cost[i,p]*Make[i,p] = 1215250

ampl: \f(CXdisplay sum {i in ORIG, j in DEST, p in PROD}
            trans_cost[i,j,p] * Trans[i,j,p];\fP
sum{i in ORIG, j in DEST, p in PROD}
   trans_cost[i,j,p]*Trans[i,j,p] = 176925
```



In [5]:
ampl.display('sum {i in ORIG, p in PROD} make_cost[i,p] * Make[i,p];')

sum{i in ORIG, p in PROD} make_cost[i,p]*Make[i,p] = 1215250



In [6]:
ampl.display('sum {i in ORIG, j in DEST, p in PROD} trans_cost[i,j,p] * Trans[i,j,p];')

sum{i in ORIG, j in DEST, p in PROD} trans_cost[i,j,p]*Trans[i,j,p] = 176925



Clearly the production costs dominate in this case.  These examples point
up the ability of `AMPL` to evaluate and display any valid expression.